# Libraries

In [46]:
import pandas as pd
import requests

from ast import literal_eval

pd.set_option('display.max_columns', None) # display all columns in DF

# Uploading data

In [56]:
df_orangebook = pd.read_excel('Searchable Database of Orange Book Drug Patents.xlsx', skiprows = 1)
df_openFDA = pd.read_csv('raw_openFDA_data.csv')  
df_medicaid = pd.read_csv('raw_medicaid_data.csv')

# Data manipulation

### OpenFDA

In [57]:
df_openFDA = df_openFDA.dropna(subset=['openfda.product_ndc'])

Clearing nda

In [58]:
df_openFDA['NDA #'] = df_openFDA['application_number'].str.replace(r'\D', '')
df_openFDA['NDA #'] = pd.to_numeric(df_openFDA['NDA #'])

Unpacking list for ndc

In [59]:
df_openFDA['openfda.product_ndc'] = df_openFDA['openfda.product_ndc'].apply(literal_eval)
df_openFDA = df_openFDA.explode('openfda.product_ndc')

Splitting ndc into labeler and product code

In [60]:
df_temp = df_openFDA['openfda.product_ndc'].str.split("-", n = 1, expand = True)
df_openFDA['labeler_code']= df_temp[0]
df_openFDA['labeler_code'] = pd.to_numeric(df_openFDA['labeler_code'])
df_openFDA['product_code']= df_temp[1]
df_openFDA['product_code'] = pd.to_numeric(df_openFDA['product_code'])

### Medicaid

In [61]:
df_medicaid['labeler_code'] = pd.to_numeric(df_medicaid['labeler_code'])
df_medicaid['product_code'] = pd.to_numeric(df_medicaid['product_code'])

# Merging data

### Orange Book and OpenFDA

In [62]:
df = pd.merge(df_openFDA, df_orangebook, on = 'NDA #', how = 'left')

### Orange Book, OpenFDA and Medicaid

In [63]:
df_consolidation_data = pd.merge(df, df_medicaid, on = ['labeler_code', 'product_code'], how = 'inner')

# Downloading output

In [ ]:
df_consolidation_data.to_csv('raw_consolidation_data.csv', index = False)

In [ ]:
len(df_consolidation_data)

In [23]:
df.head(10)

,submissions,application_number,sponsor_name,products,openfda.application_number,openfda.brand_name,openfda.generic_name,openfda.manufacturer_name,openfda.product_ndc,openfda.product_type,openfda.route,openfda.substance_name,openfda.rxcui,openfda.spl_id,openfda.spl_set_id,openfda.package_ndc,openfda.unii,openfda.nui,openfda.pharm_class_epc,openfda.pharm_class_cs,openfda.pharm_class_moa,openfda.pharm_class_pe,NDA #,Ingredient(s),Company,Product Name,Approval Date,Earliest Protection DateEarliest Prot. Date,Latest Protection DateLatest Prot. Date,dsad,Months of Additional Protection TimeMonths Add. Prot. Time,Total Number of ProtectionsNum. Prot.,Unique Patents,Number of Orphan Drug ExclusivitiesNum. ODE's,Extensions
0,"[{'submission_type': 'SUPPL', 'submission_numb...",ANDA206552,FRESENIUS KABI USA,"[{'product_number': '001', 'reference_drug': '...",['ANDA206552'],['HEPARIN SODIUM'],['HEPARIN SODIUM'],"['Fresenius Kabi USA, LLC']",76045-108,['HUMAN PRESCRIPTION DRUG'],"['INTRAVENOUS', 'SUBCUTANEOUS']",['HEPARIN SODIUM'],['1798389'],['151a54ec-f7e5-4e6d-9810-cbfa75a16a22'],['07438e10-1589-4683-9f26-e5494d076cf0'],"['76045-108-00', '76045-108-10']",['ZZ45AB24CA'],NaN,NaN,NaN,NaN,NaN,206552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'submission_type': 'SUPPL', 'submission_numb...",NDA019594,ALLERGAN,"[{'product_number': '001', 'reference_drug': '...",['NDA019594'],['URSODIOL'],['URSODIOL'],"['Actavis Pharma, Inc.']",0591-3159,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['URSODIOL'],['858747'],['9f386262-71fb-48f0-b992-773d35e952c5'],['a061bb07-a85d-4b08-877f-ce02156f0de7'],['0591-3159-01'],['724L30Y2QR'],"['N0000175802', 'M0002475']",['Bile Acid [EPC]'],['Bile Acids and Salts [CS]'],NaN,NaN,19594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'submission_type': 'SUPPL', 'submission_numb...",BLA125509,ELUSYS THERAPEUTICS INC,"[{'product_number': '001', 'reference_drug': '...",['BLA125509'],['ANTHIM'],['OBILTOXAXIMAB'],"['Elusys Therapeutics, Inc.']",69604-204,['HUMAN PRESCRIPTION DRUG'],['INTRAVENOUS'],['OBILTOXAXIMAB'],"['1746910', '1746915']",['9a6bcd22-1783-400b-b87a-c825315f600c'],['39ad8799-00a4-4fc8-9852-c0536350c474'],['69604-204-02'],['29Z5DNL48C'],"['N0000192703', 'N0000192702']",['Anthrax Protective Antigen-directed Antibody...,NaN,['Anthrax Protective Antigen-directed Antibody...,NaN,125509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'submission_type': 'ORIG', 'submission_numbe...",ANDA200884,SUN PHARM,"[{'product_number': '002', 'reference_drug': '...",['ANDA200884'],['FENOFIBRATE'],['FENOFIBRATE'],"['Sun Pharmaceutical Industries, Inc.']",63304-448,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['FENOFIBRATE'],"['477560', '477562']",['5f5e6f81-fe5a-4ad6-be4e-0cf5f28a0f00'],['125bbd7e-86a2-4209-90a5-ad353e572d1b'],"['63304-448-30', '63304-448-90', '63304-448-05...",['U202363UOS'],"['N0000175596', 'N0000175375', 'M0199111']",['Peroxisome Proliferator Receptor alpha Agoni...,['PPAR alpha [CS]'],['Peroxisome Proliferator-activated Receptor a...,NaN,200884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'submission_type': 'ORIG', 'submission_numbe...",ANDA200884,SUN PHARM,"[{'product_number': '002', 'reference_drug': '...",['ANDA200884'],['FENOFIBRATE'],['FENOFIBRATE'],"['Sun Pharmaceutical Industries, Inc.']",63304-449,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['FENOFIBRATE'],"['477560', '477562']",['5f5e6f81-fe5a-4ad6-be4e-0cf5f28a0f00'],['125bbd7e-86a2-4209-90a5-ad353e572d1b'],"['63304-448-30', '63304-448-90', '63304-448-05...",['U202363UOS'],"['N0000175596', 'N0000175375', 'M0199111']",['Peroxisome Proliferator Receptor alpha Agoni...,['PPAR alpha [CS]'],['Peroxisome Proliferator-activated Receptor a...,NaN,200884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"[{'submission_type': 'ORIG', 'submission_numbe...",ANDA201687,ANCHEN PHARMS,"[{'product_number': '001', 'reference_drug': '...",['ANDA201687'],['TRETINOIN'],['TRETINOIN'],"['Par Pharmaceutical, Inc.']",10370-268,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['T

# First figure

In [34]:
df['openfda.generic_name'] = df['openfda.generic_name'].apply(literal_eval)

In [35]:
df = df.explode('openfda.generic_name')

In [36]:
df

,submissions,application_number,sponsor_name,products,openfda.application_number,openfda.brand_name,openfda.generic_name,openfda.manufacturer_name,openfda.product_ndc,openfda.product_type,openfda.route,openfda.substance_name,openfda.rxcui,openfda.spl_id,openfda.spl_set_id,openfda.package_ndc,openfda.unii,openfda.nui,openfda.pharm_class_moa,openfda.pharm_class_epc,openfda.pharm_class_cs,openfda.pharm_class_pe,NDA #,labeler_code,product_code,Ingredient(s),Company,Product Name,Approval Date,Earliest Protection DateEarliest Prot. Date,Latest Protection DateLatest Prot. Date,dsad,Months of Additional Protection TimeMonths Add. Prot. Time,Total Number of ProtectionsNum. Prot.,Unique Patents,Number of Orphan Drug ExclusivitiesNum. ODE's,Extensions,dataset,year,quarter,price_per_unit
0,"[{'submission_type': 'SUPPL', 'submission_numb...",ANDA075967,SUN PHARM INDS INC,"[{'product_number': '003', 'reference_drug': '...",['ANDA075967'],['METFORMIN HYDROCHLORIDE'],METFORMIN HYDROCHLORIDE,"['Sun Pharmaceutical Industries, Inc.']",57664-397,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['METFORMIN HYDROCHLORIDE'],"['861004', '861007', '861010']",['7274fdaf-11f8-2b2f-e053-2991aa0ad3f6'],['9c74e4f2-b323-41eb-9e43-848ab82b527b'],"['57664-397-59', '57664-397-51', '57664-397-53...",['786Z46389E'],NaN,NaN,NaN,NaN,NaN,75967,57664,397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,State Drug Utilization Data 2002,2002,1,0.584158
1,"[{'submission_type': 'SUPPL', 'submission_numb...",ANDA075967,SUN PHARM INDS INC,"[{'product_number': '003', 'reference_drug': '...",['ANDA075967'],['METFORMIN HYDROCHLORIDE'],METFORMIN HYDROCHLORIDE,"['Sun Pharmaceutical Industries, Inc.']",57664-397,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['METFORMIN HYDROCHLORIDE'],"['861004', '861007', '861010']",['7274fdaf-11f8-2b2f-e053-2991aa0ad3f6'],['9c74e4f2-b323-41eb-9e43-848ab82b527b'],"['57664-397-59', '57664-397-51', '57664-397-53...",['786Z46389E'],NaN,NaN,NaN,NaN,NaN,75967,57664,397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,State Drug Utilization Data 2002,2002,2,0.612683
2,"[{'submission_type': 'SUPPL', 'submission_numb...",ANDA075967,SUN PHARM INDS INC,"[{'product_number': '003', 'reference_drug': '...",['ANDA075967'],['METFORMIN HYDROCHLORIDE'],METFORMIN HYDROCHLORIDE,"['Sun Pharmaceutical Industries, Inc.']",57664-397,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['METFORMIN HYDROCHLORIDE'],"['861004', '861007', '861010']",['7274fdaf-11f8-2b2f-e053-2991aa0ad3f6'],['9c74e4f2-b323-41eb-9e43-848ab82b527b'],"['57664-397-59', '57664-397-51', '57664-397-53...",['786Z46389E'],NaN,NaN,NaN,NaN,NaN,75967,57664,397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,State Drug Utilization Data 2002,2002,3,0.465164
3,"[{'submission_type': 'SUPPL', 'submission_numb...",ANDA075967,SUN PHARM INDS INC,"[{'product_number': '003', 'reference_drug': '...",['ANDA075967'],['METFORMIN HYDROCHLORIDE'],METFORMIN HYDROCHLORIDE,"['Sun Pharmaceutical Industries, Inc.']",57664-397,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['METFORMIN HYDROCHLORIDE'],"['861004', '861007', '861010']",['7274fdaf-11f8-2b2f-e053-2991aa0ad3f6'],['9c74e4f2-b323-41eb-9e43-848ab82b527b'],"['57664-397-59', '57664-397-51', '57664-397-53...",['786Z46389E'],NaN,NaN,NaN,NaN,NaN,75967,57664,397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,State Drug Utilization Data 2002,2002,4,0.478806
4,"[{'submission_type': 'SUPPL', 'submission_numb...",ANDA075967,SUN PHARM INDS INC,"[{'product_number': '003', 'reference_drug': '...",['ANDA075967'],['METFORMIN HYDROCHLORIDE'],METFORMIN HYDROCHLORIDE,"['Sun Pharmaceutical Industries, Inc.']",57664-397,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['METFORMIN HYDROCHLORIDE'],"['861004', '861007', '861010']",['7274fdaf-11f8-2b2f-e053-2991aa0ad3f6'],['9c74e4f2-b323-41eb-9e43-848ab82b527b'],"['57664-397-59', '57664-397-51', '57664-397-53...",['786Z46389E'],NaN,NaN,NaN,NaN,NaN,75967,57664,397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,State Drug Utilization Data 2003,2003,1,0.477344
...,...,...,...,...,...,...,...,...,..

In [43]:
df_test = df.loc[df['openfda.generic_name'] == 'DIMETHYL FUMARATE']

In [44]:
df_test

,submissions,application_number,sponsor_name,products,openfda.application_number,openfda.brand_name,openfda.generic_name,openfda.manufacturer_name,openfda.product_ndc,openfda.product_type,openfda.route,openfda.substance_name,openfda.rxcui,openfda.spl_id,openfda.spl_set_id,openfda.package_ndc,openfda.unii,openfda.nui,openfda.pharm_class_moa,openfda.pharm_class_epc,openfda.pharm_class_cs,openfda.pharm_class_pe,NDA #,labeler_code,product_code,Ingredient(s),Company,Product Name,Approval Date,Earliest Protection DateEarliest Prot. Date,Latest Protection DateLatest Prot. Date,dsad,Months of Additional Protection TimeMonths Add. Prot. Time,Total Number of ProtectionsNum. Prot.,Unique Patents,Number of Orphan Drug ExclusivitiesNum. ODE's,Extensions,dataset,year,quarter,price_per_unit
12002,"[{'submission_type': 'SUPPL', 'submission_numb...",NDA204063,BIOGEN INC,"[{'product_number': '001', 'reference_drug': '...",['NDA204063'],"['TECFIDERA', 'DIMETHYL FUMARATE']",DIMETHYL FUMARATE,"['Biogen Inc.', 'TEVA PHARMACEUTICALS USA, INC.']",64406-005,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['DIMETHYL FUMARATE'],"['1373483', '1373489', '1373491', '1373493', '...","['c7c11174-dd00-47fa-8c12-3692da87756b', '0b26...","['665d7e74-036c-5f68-5b67-ab84b9b49151', '8e2c...","['64406-007-03', '64406-005-01', '64406-006-02...",['FO2303MNI2'],NaN,NaN,NaN,NaN,NaN,204063,64406,5,Dimethyl Fumarate,BIOGEN IDEC INC,Tecfidera,3/27/2013,2/7/2018,2/7/2028,2028.0,120.0,19.0,7.0,0.0,2.0,State Drug Utilization Data 2013,2013,2,76.146773
12003,"[{'submission_type': 'SUPPL', 'submission_numb...",NDA204063,BIOGEN INC,"[{'product_number': '001', 'reference_drug': '...",['NDA204063'],"['TECFIDERA', 'DIMETHYL FUMARATE']",DIMETHYL FUMARATE,"['Biogen Inc.', 'TEVA PHARMACEUTICALS USA, INC.']",64406-005,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['DIMETHYL FUMARATE'],"['1373483', '1373489', '1373491', '1373493', '...","['c7c11174-dd00-47fa-8c12-3692da87756b', '0b26...","['665d7e74-036c-5f68-5b67-ab84b9b49151', '8e2c...","['64406-007-03', '64406-005-01', '64406-006-02...",['FO2303MNI2'],NaN,NaN,NaN,NaN,NaN,204063,64406,5,Dimethyl Fumarate,BIOGEN IDEC INC,Tecfidera,3/27/2013,2/7/2018,2/7/2028,2028.0,120.0,19.0,7.0,0.0,2.0,State Drug Utilization Data 2013,2013,3,78.103577
12004,"[{'submission_type': 'SUPPL', 'submission_numb...",NDA204063,BIOGEN INC,"[{'product_number': '001', 'reference_drug': '...",['NDA204063'],"['TECFIDERA', 'DIMETHYL FUMARATE']",DIMETHYL FUMARATE,"['Biogen Inc.', 'TEVA PHARMACEUTICALS USA, INC.']",64406-005,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['DIMETHYL FUMARATE'],"['1373483', '1373489', '1373491', '1373493', '...","['c7c11174-dd00-47fa-8c12-3692da87756b', '0b26...","['665d7e74-036c-5f68-5b67-ab84b9b49151', '8e2c...","['64406-007-03', '64406-005-01', '64406-006-02...",['FO2303MNI2'],NaN,NaN,NaN,NaN,NaN,204063,64406,5,Dimethyl Fumarate,BIOGEN IDEC INC,Tecfidera,3/27/2013,2/7/2018,2/7/2028,2028.0,120.0,19.0,7.0,0.0,2.0,State Drug Utilization Data 2013,2013,4,75.998446
12005,"[{'submission_type': 'SUPPL', 'submission_numb...",NDA204063,BIOGEN INC,"[{'product_number': '001', 'reference_drug': '...",['NDA204063'],"['TECFIDERA', 'DIMETHYL FUMARATE']",DIMETHYL FUMARATE,"['Biogen Inc.', 'TEVA PHARMACEUTICALS USA, INC.']",64406-005,['HUMAN PRESCRIPTION DRUG'],['ORAL'],['DIMETHYL FUMARATE'],"['1373483', '1373489', '1373491', '1373493', '...","['c7c11174-dd00-47fa-8c12-3692da87756b', '0b26...","['665d7e74-036c-5f68-5b67-ab84b9b49151', '8e2c...","['64406-007-03', '64406-005-01', '64406-006-02...",['FO2303MNI2'],NaN,NaN,NaN,NaN,NaN,204063,64406,5,Dimethyl Fumarate,BIOGEN IDEC INC,Tecfidera,3/27/2013,2/7/2018,2/7/2028,2028.0,120.0,19.0,7.0,0.0,2.0,State Drug Utilization Data 2014,2014,1,80.725235
12006,"[{'submission_type': 'SUPPL', 'submission_numb...",NDA204063,BIOGEN INC,"[{'product_number': '001', 'reference_drug': '...",['NDA204063'],"['TECFIDERA', 'DIMETHYL FUMARATE']",DIMETHYL FUMARATE,"['Biogen Inc.', 'TEVA PHARMACEUTICALS USA, INC.']",64406-005,['HUMAN PRESCRIPTION DRUG']

In [45]:
df_test.to_excel('test_output.xlsx', index = False)